In [2]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
import numpy as np
import pandas as pd
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [3]:
df=pd.read_csv('fra.txt', delimiter='\t')
df

,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
0,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
227809,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
227810,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
227811,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
227812,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [4]:
batch_size=16
epochs=100
latent_dim=256
num_samples=10000

In [6]:
file_path='fra.txt'
# vectorize
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(file_path, 'r', encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _= line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [7]:
input_texts

['Go.',
 'Go.',
 'Go.',
 'Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Who?',
 'Wow!',
 'Wow!',
 'Wow!',
 'Duck!',
 'Duck!',
 'Duck!',
 'Fire!',
 'Help!',
 'Hide.',
 'Hide.',
 'Jump!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Begin.',
 'Begin.',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'Hello.',
 'Hello.',
 'Hello.',
 'Hello.',
 'I see.',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Smile.',
 'Smile.',
 'Smile.',
 'Sorry?',
 'Attack!',
 'Attack!',
 'Attack!',
 'Attack!',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Eat it.',
 'Eat it.',
 'Exhale.',
 'Get up.',
 'Get up.',
 'Ge

In [8]:
target_texts

['\tVa !\n',
 '\tMarche.\n',
 '\tEn route !\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tWaouh\xa0!\n',
 '\tWah\xa0!\n',
 '\tÀ terre\xa0!\n',
 '\tBaisse-toi\xa0!\n',
 '\tBaissez-vous\xa0!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tCache-toi.\n',
 '\tCachez-vous.\n',
 '\tSaute.\n',
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttendez.\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttends.\n',
 '\tAttendez.\n',
 '\tCommencez.\n',
 '\tCommence.\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\t

In [9]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
    return clean_sentence

In [10]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [11]:
import string
target_texts= [clean_sentence(i) for i in target_texts]
input_texts= [clean_sentence(i) for i in input_texts]

In [12]:
target_texts

['\tva \n',
 '\tmarche\n',
 '\ten route \n',
 '\tbouge \n',
 '\tsalut \n',
 '\tsalut\n',
 '\tcours\u202f\n',
 '\tcourez\u202f\n',
 '\tprenez vos jambes à vos cous \n',
 '\tfile \n',
 '\tfilez \n',
 '\tcours \n',
 '\tfuyez \n',
 '\tfuyons \n',
 '\tcours\u202f\n',
 '\tcourez\u202f\n',
 '\tprenez vos jambes à vos cous \n',
 '\tfile \n',
 '\tfilez \n',
 '\tcours \n',
 '\tfuyez \n',
 '\tfuyons \n',
 '\tqui \n',
 '\tça alors\u202f\n',
 '\twaouh\xa0\n',
 '\twah\xa0\n',
 '\tà terre\xa0\n',
 '\tbaissetoi\xa0\n',
 '\tbaissezvous\xa0\n',
 '\tau feu \n',
 '\tà laide\u202f\n',
 '\tcachetoi\n',
 '\tcachezvous\n',
 '\tsaute\n',
 '\tsaute\n',
 '\tça suffit\u202f\n',
 '\tstop\u202f\n',
 '\tarrêtetoi \n',
 '\tattends \n',
 '\tattendez \n',
 '\tattendez\n',
 '\tattends \n',
 '\tattendez \n',
 '\tattends\n',
 '\tattendez\n',
 '\tcommencez\n',
 '\tcommence\n',
 '\tpoursuis\n',
 '\tcontinuez\n',
 '\tpoursuivez\n',
 '\tbonjour \n',
 '\tsalut \n',
 '\tbonjour \n',
 '\tsalut \n',
 '\tbonjour\n',
 '\tsalut\n',


In [13]:
input_texts

['go',
 'go',
 'go',
 'go',
 'hi',
 'hi',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'run',
 'who',
 'wow',
 'wow',
 'wow',
 'duck',
 'duck',
 'duck',
 'fire',
 'help',
 'hide',
 'hide',
 'jump',
 'jump',
 'stop',
 'stop',
 'stop',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'begin',
 'begin',
 'go on',
 'go on',
 'go on',
 'hello',
 'hello',
 'hello',
 'hello',
 'hello',
 'hello',
 'i see',
 'i see',
 'i try',
 'i won',
 'i won',
 'i won',
 'oh no',
 'relax',
 'relax',
 'relax',
 'relax',
 'relax',
 'relax',
 'relax',
 'relax',
 'relax',
 'relax',
 'relax',
 'relax',
 'smile',
 'smile',
 'smile',
 'sorry',
 'attack',
 'attack',
 'attack',
 'attack',
 'buy it',
 'buy it',
 'buy it',
 'buy it',
 'cheers',
 'cheers',
 'cheers',
 'cheers',
 'eat it',
 'eat it',
 'exhale',
 'get up',
 'get up',
 'get up',
 'go now',
 'go now',
 'go now',
 'got it',
 'got it',
 'got it',
 'got it',
 'got it',
 

In [14]:
# Tokenize words
fre_text_tokenized, fre_text_tokenizer = tokenize(target_texts)
eng_text_tokenized, eng_text_tokenizer = tokenize(input_texts)

In [15]:
fre_text_tokenized[16]

[95, 275, 512, 14, 275, 944]

In [16]:
eng_text_tokenized[500]

[26, 1, 10]

In [17]:
# Check language length
fre_vocab = len(fre_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1

In [18]:
english_vocab

2028

In [19]:
fre_vocab

4697

In [20]:
max_french_len = int(len(max(fre_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

In [21]:
max_french_len

10

In [22]:
max_english_len

4

In [23]:
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")
fre_pad_sentence = pad_sequences(fre_text_tokenized, max_french_len, padding = "post")

In [24]:
eng_pad_sentence.shape

(10000, 4)

In [25]:
fre_pad_sentence.shape

(10000, 10)

In [26]:
# Reshape data
fre_pad_sentence = fre_pad_sentence.reshape(*fre_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

print(fre_pad_sentence.shape)
print(eng_pad_sentence.shape)

(10000, 10, 1)
(10000, 4, 1)


encoder

In [27]:
input_sequence = Input(shape=(max_english_len,))
print(input_sequence.shape)

(None, 4)


In [29]:
embedding = Embedding(input_dim=english_vocab, output_dim=128,)(input_sequence)
print(embedding.shape)

(None, 4, 128)


In [30]:
encoder = LSTM(64, return_sequences=False)(embedding)
print(encoder.shape)

(None, 64)


decoder

In [31]:
r_vec = RepeatVector(max_french_len)(encoder)
print(r_vec.shape)

(None, 10, 64)


In [32]:
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
print(decoder.shape)

(None, 10, 64)


In [33]:
logits = TimeDistributed(Dense(fre_vocab))(decoder)
logits

<KerasTensor: shape=(None, 10, 4697) dtype=float32 (created by layer 'time_distributed')>

In [34]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model

In [35]:
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 embedding (Embedding)       (None, 4, 128)            259584    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVecto  (None, 10, 64)            0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 10, 64)            33024     
                                                                 
 time_distributed (TimeDist  (None, 10, 4697)          305305    
 ributed)                                                    

In [36]:
model_results = enc_dec_model.fit(eng_pad_sentence, fre_pad_sentence, batch_size=30, epochs=50)

Epoch 1/50
334/334 [==============================] - 22s 33ms/step - loss: 2.9997 - accuracy: 0.6964
Epoch 2/50
334/334 [==============================] - 11s 33ms/step - loss: 2.0706 - accuracy: 0.7162
Epoch 3/50
334/334 [==============================] - 11s 33ms/step - loss: 1.9568 - accuracy: 0.7233
Epoch 4/50
334/334 [==============================] - 11s 34ms/step - loss: 1.8995 - accuracy: 0.7252
Epoch 5/50
334/334 [==============================] - 11s 33ms/step - loss: 1.8600 - accuracy: 0.7264
Epoch 6/50
334/334 [==============================] - 11s 33ms/step - loss: 1.8239 - accuracy: 0.7283
Epoch 7/50
334/334 [==============================] - 11s 33ms/step - loss: 1.7881 - accuracy: 0.7313
Epoch 8/50
334/334 [==============================] - 11s 33ms/step - loss: 1.7449 - accuracy: 0.7362
Epoch 9/50
334/334 [==============================] - 11s 33ms/step - loss: 1.6899 - accuracy: 0.7436
Epoch 10/50
334/334 [==============================] - 11s 33ms/step - loss: 1.636

In [37]:
# model_results = enc_dec_model.fit(fre_pad_sentence, eng_pad_sentence, batch_size=30, epochs=50)

In [40]:
def logits_to_sentence(logits, tokenizer):
    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = ' ' 
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [44]:
index = 131

print("The english sentence is: {}".format(input_texts[index]))
print("The french sentence is: {}".format(target_texts[index]))

print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(eng_pad_sentence[index:index+1])[0], fre_text_tokenizer))

The english sentence is: listen
The french sentence is: 	écoutez 

The predicted sentence is :
1/1 [==============================] - 0s 16ms/step
écoutez                  
